In [1]:
from __future__ import annotations

import logging

import rich
from rich import print as rprint
from rich.logging import RichHandler

logging.basicConfig(
    level="INFO",
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True, enable_link_path=False)],
)

In [12]:
from abc import ABC
from collections.abc import Sequence
from typing import Annotated, Literal

from typing_extensions import TypeAliasType

import nshconfig as C


# This test is a simplified version of the one in nshtrainer,
# which currently fails with pydantic>=2.11, but works with pydantic<2.11.
# Track the following issue: https://github.com/pydantic/pydantic/issues/11682
class PluginBaseConfig(C.Config, ABC):
    pass


plugin_registry = C.Registry(
    PluginBaseConfig,
    discriminator="name",
    config={
        "duplicate_tag_policy": "warn-and-replace",
        "auto_rebuild": False,
    },
)
PluginConfig = TypeAliasType(
    "PluginConfig", Annotated[PluginBaseConfig, plugin_registry.DynamicResolution()]
)
rprint(plugin_registry)
rprint(plugin_registry.pydantic_schema())


@plugin_registry.rebuild_on_registers
class RootConfig(C.Config):
    plugins: Sequence[PluginConfig] | None = None


rprint(RootConfig(plugins=[]))
rprint(plugin_registry)


Registry(
    base_cls=<class '__main__.PluginBaseConfig'>,
    discriminator='name',
    config={'duplicate_tag_policy': 'warn-and-replace', 'auto_rebuild': False},
    _elements=[],
    _on_register_callbacks=[]
)

{
    'type': 'model',
    'cls': <class 'nshconfig._src.invalid.Invalid'>,
    'schema': {
        'type': 'function-before',
        'function': {
            'type': 'no-info',
            'function': <bound method Invalid.invalidate of <class 'nshconfig._src.invalid.Invalid'>>
        },
        'schema': {'type': 'model-fields', 'fields': {}, 'model_name': 'Invalid', 'computed_fields': []}
    },
    'custom_init': False,
    'root_model': False,
    'post_init': 'model_post_init',
    'config': {
        'title': 'Invalid',
        'extra_fields_behavior': 'ignore',
        'strict': True,
        'revalidate_instances': 'always',
        'validate_default': True,
        'validation_error_cause': True
    },
    'serialization': {
        'type': 'function-wrap',
        'function': <function Config.include_literals at 0x7e07b214c220>,
        'info_arg': False
    },
    'ref': 'nshconfig._src.invalid.Invalid:150971488',
    'metadata': {
        'pydantic_js_functions': [
            <bound method BaseModel.__get_pydantic_json_schema__ of <class 'nshconfig._src.invalid.Invalid'>>
        ]
    }
}

[17:06:42] INFO     Rebuilt <class '__main__.RootConfig'> schema due to registry changes.           registry.py:361

RootConfig(plugins=[])

Registry(
    base_cls=<class '__main__.PluginBaseConfig'>,
    discriminator='name',
    config={'duplicate_tag_policy': 'warn-and-replace', 'auto_rebuild': False},
    _elements=[],
    _on_register_callbacks=[<function Registry.rebuild_on_registers.<locals>._rebuild at 0x7e07a6f92d40>]
)

In [13]:
@plugin_registry.register
class Plugin1(PluginBaseConfig):
    name: Literal["plugin1"] = "plugin1"
    value: int = 42


print(plugin_registry)
rprint(plugin_registry.pydantic_schema())


print(RootConfig(plugins=[]))
print(RootConfig(plugins=[Plugin1()]))

[17:06:43] INFO     Registered <class '__main__.Plugin1'> with tag 'plugin1'.                       registry.py:257

           INFO     Rebuilt <class '__main__.RootConfig'> schema due to registry changes.           registry.py:361

Registry(base_cls=<class '__main__.PluginBaseConfig'>, discriminator='name', config={'duplicate_tag_policy': 'warn-and-replace', 'auto_rebuild': False}, _elements=[_RegistryEntry(tag='plugin1', cls=<class '__main__.Plugin1'>)], _on_register_callbacks=[<function Registry.rebuild_on_registers.<locals>._rebuild at 0x7e07a6f92d40>])


{
    'type': 'tagged-union',
    'choices': {
        'plugin1': {
            'type': 'model',
            'cls': <class '__main__.Plugin1'>,
            'schema': {
                'type': 'model-fields',
                'fields': {
                    'name': {
                        'type': 'model-field',
                        'schema': {
                            'type': 'default',
                            'schema': {'type': 'literal', 'expected': ['plugin1']},
                            'default': 'plugin1'
                        },
                        'metadata': {}
                    },
                    'value': {
                        'type': 'model-field',
                        'schema': {'type': 'default', 'schema': {'type': 'int'}, 'default': 42},
                        'metadata': {}
                    }
                },
                'model_name': 'Plugin1',
                'computed_fields': []
            },
            'custom_init': False,
            'root_model': False,
            'post_init': 'model_post_init',
            'config': {
                'title': 'Plugin1',
                'extra_fields_behavior': 'ignore',
                'strict': True,
                'revalidate_instances': 'always',
                'validate_default': True,
                'validation_error_cause': True
            },
            'serialization': {
                'type': 'function-wrap',
                'function': <function Config.include_literals at 0x7e07b214c220>,
                'info_arg': False
            },
            'ref': '__main__.Plugin1:151120240',
            'metadata': {
                'pydantic_js_functions': [
                    <bound method BaseModel.__get_pydantic_json_schema__ of <class '__main__.Plugin1'>>
                ]
            }
        }
    },
    'discriminator': 'name',
    'strict': False,
    'from_attributes': True,
    'metadata': {}
}

plugins=[]
plugins=[Plugin1(name='plugin1', value=42)]


In [14]:
RootConfig.model_rebuild()
rprint(RootConfig.__pydantic_core_schema__)

{
    'type': 'definitions',
    'schema': {
        'type': 'model',
        'cls': <class '__main__.RootConfig'>,
        'schema': {
            'type': 'model-fields',
            'fields': {
                'plugins': {
                    'type': 'model-field',
                    'schema': {
                        'type': 'default',
                        'schema': {
                            'type': 'nullable',
                            'schema': {
                                'type': 'json-or-python',
                                'json_schema': {
                                    'type': 'list',
                                    'items_schema': {
                                        'type': 'definition-ref',
                                        'schema_ref': '__main__.PluginConfig:138571509648368'
                                    }
                                },
                                'python_schema': {
                                    'type': 'chain',
                                    'steps': [
                                        {'type': 'is-instance', 'cls': typing.Sequence, 'cls_repr': 'Sequence'},
                                        {
                                            'type': 'function-wrap',
                                            'function': {
                                                'type': 'no-info',
                                                'function': <function sequence_validator at 0x7e07b2b4e340>
                                            },
                                            'schema': {
                                                'type': 'list',
                                                'items_schema': {
                                                    'type': 'definition-ref',
                                                    'schema_ref': '__main__.PluginConfig:138571509648368'
                                                }
                                            }
                                        }
                                    ]
                                },
                                'serialization': {
                                    'type': 'function-wrap',
                                    'function': <function serialize_sequence_via_list at 0x7e07b21f85e0>,
                                    'info_arg': True,
                                    'schema': {
                                        'type': 'definition-ref',
                                        'schema_ref': '__main__.PluginConfig:138571509648368'
                                    }
                                }
                            }
                        },
                        'default': None
                    },
                    'metadata': {}
                }
            },
            'model_name': 'RootConfig',
            'computed_fields': []
        },
        'custom_init': False,
        'root_model': False,
        'post_init': 'model_post_init',
        'config': {
            'title': 'RootConfig',
            'extra_fields_behavior': 'ignore',
            'strict': True,
            'revalidate_instances': 'always',
            'validate_default': True,
            'validation_error_cause': True
        },
        'serialization': {
            'type': 'function-wrap',
            'function': <function Config.include_literals at 0x7e07b214c220>,
            'info_arg': False
        },
        'ref': '__main__.RootConfig:138570543160448',
        'metadata': {
            'pydantic_js_functions': [
                <bound method BaseModel.__get_pydantic_json_schema__ of <class '__main__.RootConfig'>>
            ]
        }
    },
    'definitions': [
        {
            'type': 'tagged-union',
            'choices': {
                'plugin1': {
                    'type': 'model',
                    'cls': <class '__main__.Plugin1'>,
              

In [ ]:
from typing import Any, TypeAlias

A: TypeAlias = Annotated[PluginBaseConfig, plugin_registry.DynamicResolution()]


@plugin_registry.register
class Plugin2(PluginBaseConfig):
    name: Literal["plugin2"] = "plugin2"

    nested_plugin: A | None = None


rprint(plugin_registry)
rprint(plugin_registry.pydantic_schema())

rprint(RootConfig(plugins=[]))
rprint(RootConfig(plugins=[Plugin1()]))
rprint(RootConfig(plugins=[Plugin2()]))

RecursionError: maximum recursion depth exceeded

In [5]:
PluginBaseConfig.__pydantic_core_schema__

{'type': 'model',
 'cls': __main__.PluginBaseConfig,
 'schema': {'type': 'model-fields',
  'fields': {},
  'model_name': 'PluginBaseConfig',
  'computed_fields': []},
 'custom_init': False,
 'root_model': False,
 'post_init': 'model_post_init',
 'config': {'title': 'PluginBaseConfig',
  'extra_fields_behavior': 'ignore',
  'strict': True,
  'revalidate_instances': 'always',
  'validate_default': True,
  'validation_error_cause': True},
 'serialization': {'type': 'function-wrap',
  'function': <function nshconfig._src.config.Config.include_literals(self, next_serializer)>,
  'info_arg': False},
 'ref': '__main__.PluginBaseConfig:944843392',
 'metadata': {'pydantic_js_functions': [<bound method BaseModel.__get_pydantic_json_schema__ of <class '__main__.PluginBaseConfig'>>]}}

In [6]:
C.TypeAdapter(PluginBaseConfig).core_schema

{'type': 'model',
 'cls': __main__.PluginBaseConfig,
 'schema': {'type': 'model-fields',
  'fields': {},
  'model_name': 'PluginBaseConfig',
  'computed_fields': []},
 'custom_init': False,
 'root_model': False,
 'post_init': 'model_post_init',
 'config': {'title': 'PluginBaseConfig',
  'extra_fields_behavior': 'ignore',
  'strict': True,
  'revalidate_instances': 'always',
  'validate_default': True,
  'validation_error_cause': True},
 'serialization': {'type': 'function-wrap',
  'function': <function nshconfig._src.config.Config.include_literals(self, next_serializer)>,
  'info_arg': False},
 'ref': '__main__.PluginBaseConfig:944843392',
 'metadata': {'pydantic_js_functions': [<bound method BaseModel.__get_pydantic_json_schema__ of <class '__main__.PluginBaseConfig'>>]}}

In [7]:
from pydantic._internal import (
    _config,
    _generate_schema,
    _mock_val_ser,
    _namespace_utils,
    _repr,
    _typing_extra,
    _utils,
)

localns = {}
globalns = {}
ns_resolver = _namespace_utils.NsResolver(
    namespaces_tuple=_namespace_utils.NamespacesTuple(locals=localns, globals=globalns),
    parent_namespace=localns,
)
config_wrapper = _config.ConfigWrapper({})
schema_generator = _generate_schema.GenerateSchema(
    config_wrapper, ns_resolver=ns_resolver
)
core_schema = schema_generator.generate_schema(PluginBaseConfig)
pprint(core_schema)

core_schema = schema_generator.clean_schema(core_schema)
pprint(core_schema)
pprint(PluginBaseConfig.__pydantic_core_schema__)

{'schema_ref': '__main__.PluginBaseConfig:944843392', 'type': 'definition-ref'}
{'cls': <class '__main__.PluginBaseConfig'>,
 'config': {'extra_fields_behavior': 'ignore',
            'revalidate_instances': 'always',
            'strict': True,
            'title': 'PluginBaseConfig',
            'validate_default': True,
            'validation_error_cause': True},
 'custom_init': False,
 'metadata': {'pydantic_js_functions': [<bound method BaseModel.__get_pydantic_json_schema__ of <class '__main__.PluginBaseConfig'>>]},
 'post_init': 'model_post_init',
 'ref': '__main__.PluginBaseConfig:944843392',
 'root_model': False,
 'schema': {'computed_fields': [],
            'fields': {},
            'model_name': 'PluginBaseConfig',
            'type': 'model-fields'},
 'serialization': {'function': <function Config.include_literals at 0x7eccac0653a0>,
                   'info_arg': False,
                   'type': 'function-wrap'},
 'type': 'model'}
{'cls': <class '__main__.PluginBaseConf